## 0. Setup

In [25]:
!pip install PyPDF2 langchain tiktoken chromadb openai pytest pytest-mock pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.1/997.1 kB 9.1 MB/s  0:00:00


## 1. Extracting files

In [ ]:
# pdf_loader.py
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extrae todo el texto de un archivo PDF.
    Args:
      pdf_path (str): Ruta al archivo PDF.
    Returns:
      str: Texto completo extraído.
    """
    reader = PdfReader(pdf_path)
    full_text = []
    for page in reader.pages:
        full_text.append(page.extract_text())
    return "\n".join(filter(None, full_text))


# Test de unidad básico
if __name__ == "__main__":
    example_pdf_path = "./test_docs/manual_empleado.pdf"
    text = extract_text_from_pdf(example_pdf_path)
    print(f"Extracted text length: {len(text)}")

Extracted text length: 249017


## 2. Text Splitting into Chunks

In [37]:
# chunking.py
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List

def chunk_text(text: str, chunk_size: int = 1000, chunk_overlap: int = 100) -> List[str]:
    """
    Divide el texto en chunks compatibles con LangChain.
    
    Args:
      text (str): Texto completo a dividir.
      chunk_size (int): Tamaño máximo por chunk.
      chunk_overlap (int): Tamaño de overlap para mantener contexto.
      
    Returns:
      List[str]: Lista de fragmentos de texto.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)


# Test unidad básico
if __name__ == "__main__":
    pdf_path = "./test_docs/manual_empleado.pdf"
    chunks = chunk_text(text)
    print(f"Chunks created: {len(chunks)}")
    print(f"First chunk preview: {chunks[0][:2000]}")

Chunks created: 282
First chunk preview: Aprobado por la
 Junta de Síndicos 
Voto: JS08-202010MANUAL
DEL 
EMPLEADO
Bienvenida  .....................................................................................................................................  i
Preámbulo  .....................................................................................................................................  ii
Introducción  .................................................................................................................................  iii
Misión  ..........................................................................................................................................  iv
Visión  .............. ............................................................................................................................  iv
Objetivos  ............................................................................................................................

## 3. Embedding

In [ ]:
# embeddings.py
import os
from langchain.embeddings import OpenAIEmbeddings
import tiktoken

def batch_texts_by_token_limit(texts: list[str], max_tokens: int = 300000, model_name: str = "text-embedding-3-small") -> list[list[str]]:
    """
    Divide la lista de textos en lotes para no exceder el límite de tokens por solicitud.
    """
    enc = tiktoken.encoding_for_model(model_name)
    batches = []
    current_batch = []
    current_tokens = 0
    for text in texts:
        tokens = len(enc.encode(text))
        if current_tokens + tokens > max_tokens and current_batch:
            batches.append(current_batch)
            current_batch = []
            current_tokens = 0
        current_batch.append(text)
        current_tokens += tokens
    if current_batch:
        batches.append(current_batch)
    return batches

def create_openai_embeddings(texts: list[str]) -> list[list[float]]:
    """
    Crea embeddings usando el modelo text-embedding-3-small de OpenAI, respetando el límite de tokens por solicitud.
    Args:
      texts (list[str]): Lista de textos a vectorizar.
      openai_api_key (str): API Key OpenAI.
    Returns:
      list[list[float]]: Vectores embedding.
    """
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
    batches = batch_texts_by_token_limit(texts)
    vectors = []
    for batch in batches:
        vectors.extend(embedding_model.embed_documents(batch))
    return vectors


# Test básico
if __name__ == "__main__":
    embeddings = create_openai_embeddings(chunks)
    print(f"Embedding vector length for first text: {len(embeddings[0])}")

Embedding vector length for first text: 1536
Embedding vector length for first text: [0.04505504854388606, 0.04698253617589357, 0.06235425805052792, -0.005029540717322104, 0.01339604667929761, -0.003577900965823686, 0.00401158600898829, 0.023141910349285724, -0.04883774572253637, 0.0010586128775813598, 0.05517436419986116, -0.0043669666109888705, 0.0070774972576526955, -0.012986454719305663, 0.007282293237648669, 0.041874689853254175, 0.006354689395649879, -0.00027425303095765123, 0.01184803203397138, 0.029683324806606484, 0.012070897535308529, 0.06505274297051271, -0.02231068070262279, 0.014925990871960053, 0.002736130376993975, -0.011558908516641207, -0.026647531599930135, -0.01722692955863193, 0.018202720963959952, -0.012149202209303206, 0.016973946946629338, -0.03120122234126727, -0.01111920037730643, 0.0066076720076524735, 0.009083290029316187, 0.004981353619654179, -0.04760896984256053, 0.01756423970796872, 0.04310346573323002, -0.00986030924931037, 0.00019105478049961358, 0.0097

## 4. Vector Stores

In [48]:
# vector_store.py
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import os

def save_embeddings_in_chromadb(
    texts: list[str],
    metadatas: list[dict],
    persist_directory: str
):
    """
    Guarda fragmentos y embeddings en ChromaDB para búsqueda.
    Args:
      texts: Lista de documentos/textos.
      metadatas: Lista de diccionarios con metadatos, mismo orden que texts.
      persist_directory (str): Carpeta donde persistir la DB.
      openai_api_key (str): API Key OpenAI para crear embeddings.
    """
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = Chroma(
        collection_name="internal_docs",
        embedding_function=embedding_model,
        persist_directory=persist_directory
    )
    vectorstore.add_texts(texts=texts, metadatas=metadatas)
    vectorstore.persist()
    vectorstore = None  # Liberar memoria


# Test básico
if __name__ == "__main__":
    sample_metadata = [{"source": example_pdf_path}]

    batches = batch_texts_by_token_limit(chunks)
    for batch in batches:
        save_embeddings_in_chromadb(batch, sample_metadata, "./db")
    print("Embeddings guardados con éxito.")


/var/folders/cn/lj16zlj56_v83zrg9_b4r9gr0000gn/T/ipykernel_4022/1687061784.py:30: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


Embeddings guardados con éxito.


## 5. Retriving from the Persistant Vector Datastore

In [54]:
# retriever.py
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from chromadb.config import Settings
import chromadb

def load_retriever(persist_directory: str, openai_api_key: str):
    """
    Carga la base ChromaDB como vectorstore para realizar consultas.
    Retorna el objeto retriever.
    """
    os.environ["OPENAI_API_KEY"] = openai_api_key
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = Chroma(
        collection_name="internal_docs",
        embedding_function=embedding_model,
        persist_directory=persist_directory
    )
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
    return retriever


# Test básico: recuperar textos similares
if __name__ == "__main__":
    retriever = load_retriever("./db", os.getenv("OPENAI_API_KEY"))
    query = "¿Cuáles son las políticas de seguridad?"
    docs = retriever.get_relevant_documents(query)
    print(f"Documentos recuperados: {len(docs)}")
    print(f"Primera doc preview: {docs[0].page_content[:300] if docs else 'Sin resultados'}")

Documentos recuperados: 4
Primera doc preview: I.  POLÍTICAS
 1. Políticas para Mantener un Ambiente de Trabajo Libre de Drogas y Alcohol  .....................  1
 2. Política que Prohíbe el uso de Tabaco  ..................................................................................  2
 3. Política sobre Igualdad de Oportunidades en el Emp


## 6. Retrivers in Langchain

In [58]:
# rag_answer.py
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os

def generate_answer_from_rag(question: str, retriever, openai_api_key: str) -> str:
    """
    Dada una pregunta y un retriever, genera una respuesta usando RAG y modelo GPT.
    
    Args:
      question (str): Pregunta a responder.
      retriever: Retriever previamente inicializado (ChromaDB).
      openai_api_key (str): API key para OpenAI.
      
    Returns:
      str: Respuesta generada.
    """
    os.environ["OPENAI_API_KEY"] = openai_api_key
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    result = qa_chain.run(question)
    return result


# Test básico
if __name__ == "__main__":
    openai_api_key = os.getenv("OPENAI_API_KEY")
    retriever = load_retriever("./db", openai_api_key)
    question = "¿Qué procedimientos debo seguir en caso de emergencia laboral?"
    answer = generate_answer_from_rag(question, retriever, openai_api_key)
    print(f"Respuesta:\n{answer}")

/var/folders/cn/lj16zlj56_v83zrg9_b4r9gr0000gn/T/ipykernel_4022/404877734.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


Respuesta:
Para saber qué procedimientos seguir en caso de emergencia laboral, debes revisar el documento proporcionado en busca de secciones específicas relacionadas con emergencias laborales, seguridad en el trabajo o protocolos de emergencia. En este caso, parece que el documento proporcionado no contiene información detallada sobre procedimientos específicos para emergencias laborales. Por lo tanto, te recomendaría consultar directamente con tu empleador o el departamento de recursos humanos para obtener información detallada sobre los procedimientos a seguir en caso de emergencia en tu lugar de trabajo.
